In [1]:
import polars as pl
from datetime import datetime,timedelta
import get_and_store as gas
import polars_indicators as pi
from polars_indicators.indicators import IndicatorResult
from importlib import reload
DATE_COLUNN = "Date"

In [2]:
reload(pi)
reload(pi.strategies.buy_x_week_low)
reload(pi.summarize.trades)
reload(pi.indicators)

<module 'polars_indicators.indicators' from '/home/avery/dev/polar-indicators/src/polars_indicators/indicators/__init__.py'>

In [3]:
symbols = ['A','AA']

In [4]:
#df = gas.get_symbols_df(symbols)

In [5]:
try:
    df = saved_df.clone()
except NameError:
        df = gas.get_all_saved_data_df()
        df = df.filter(pl.col(pi.indicators.DATE_COLUMN) >= datetime(2020, 1, 1))
        df = df.collect()

In [6]:
saved_df = df.clone()

In [7]:
sma100 = pi.indicators.simple_moving_average(df, 100)
df = sma100.df
slope_up = pi.indicators.slope_up(df, column=sma100.column)
df = slope_up.df.with_columns(pl.col(slope_up.column).shift()) #shifting so we use previous bar

In [8]:
df = df.with_columns(pl.col(pi.indicators.DATE_COLUMN).cast(pl.Datetime))
df

Date,Open,High,Low,Close,Adj Close,Volume,Symbol,SMA100,SMA100_slope_up
datetime[μs],f64,f64,f64,f64,f64,i64,str,f64,bool
2020-01-02 00:00:00,4.61,4.61,4.26,4.56,4.56,116200,"""CEMI""",null,null
2020-01-03 00:00:00,5.0,5.4,4.78,5.19,5.19,432600,"""CEMI""",null,null
2020-01-06 00:00:00,5.08,5.11,4.85,4.89,4.89,140400,"""CEMI""",null,null
2020-01-07 00:00:00,4.83,5.19,4.82,5.09,5.09,75200,"""CEMI""",null,null
2020-01-08 00:00:00,5.05,5.1,4.82,4.87,4.87,71300,"""CEMI""",null,null
2020-01-09 00:00:00,4.85,4.94,4.85,4.86,4.86,21500,"""CEMI""",null,null
2020-01-10 00:00:00,4.85,4.85,3.89,4.54,4.54,557100,"""CEMI""",null,null
2020-01-13 00:00:00,4.5,4.5,4.21,4.24,4.24,87500,"""CEMI""",null,null
2020-01-14 00:00:00,4.25,4.34,4.0,4.16,4.16,60800,"""CEMI""",null,null


In [9]:
#filter out average price too low
df = df.filter(pl.col(pi.indicators.CLOSE_COLUMN).mean().over(pi.indicators.SYMBOL_COLUMN) > 3)

# filter out too many 0 volume bars. This gets rid of a lot of garbage
df = df.filter((pl.col(pi.indicators.VOLUMNE_COLUMN) == 0).sum().over(pi.indicators.SYMBOL_COLUMN) < 3)

# minimum amount of volume
df = df.filter(pl.col(pi.indicators.VOLUMNE_COLUMN).mean().over(pi.indicators.SYMBOL_COLUMN) > 100000)


In [10]:
#r = df.select(pl.col(pi.indicators.CLOSE_COLUMN) > 3)
r = df[pi.indicators.CLOSE_COLUMN] > 3
type(r)

polars.series.series.Series

In [11]:
lookback = timedelta(weeks=52)
modresult_05 = pi.strategies.buy_x_week_low.strategy(df, lookback, 0.5, 5, 2, 3)
modresult_1 = pi.strategies.buy_x_week_low.strategy(df, lookback, 1, 5, 2, 3)

The predicate 'col("364 days, 0:00:00_week_min_targets").is_not_null()' in 'when->then->otherwise' is not a valid aggregation and might produce a different number of rows than the groupby operation would. This behavior is experimental and may be subject to change
The predicate 'col("364 days, 0:00:00_week_min_targets").is_not_null()' in 'when->then->otherwise' is not a valid aggregation and might produce a different number of rows than the groupby operation would. This behavior is experimental and may be subject to change


In [12]:
trades_05 = pi.summarize.trades.Trades.from_strategy_result(modresult_05, [slope_up.column])
trades_1 = pi.summarize.trades.Trades.from_strategy_result(modresult_1, [slope_up.column])

In [13]:
trades_05.summarize_strategy(slope_up.column)

The predicate '[(col("Gain/Loss%")) > (0.0)]' in 'when->then->otherwise' is not a valid aggregation and might produce a different number of rows than the groupby operation would. This behavior is experimental and may be subject to change
The predicate '[(col("Gain/Loss%")) < (0.0)]' in 'when->then->otherwise' is not a valid aggregation and might produce a different number of rows than the groupby operation would. This behavior is experimental and may be subject to change


SMA100_slope_up,Gain/Loss%Sum,Winners,Losers,Trades,Win_Ratio,Average_Gain/Loss%,Average_Length,highest_win,Highest%,lowest_loss,Lowest%
bool,f64,u32,u32,u32,f64,f64,f64,f64,f64,f64,f64
false,66528.23,26676,15869,42563,0.626742,1.563053,4.848108,417.25,462.63,-72.26,-86.47
null,415.0,83,0,83,1.0,5.0,1.0,5.0,30.08,null,-13.25
true,800.0,160,0,160,1.0,5.0,1.0,5.0,560.43,null,-49.11


In [14]:
trades_1.summarize_strategy()

Gain/Loss%Sum,Winners,Losers,Trades,Win_Ratio,Average_Gain/Loss%,Average_Length,highest_win,Highest%,lowest_loss,Lowest%
f64,u32,u32,u32,f64,f64,f64,f64,f64,f64,f64
69558.68,28368,17906,46291,0.612819,1.502639,4.742261,414.69,557.16,-71.98,-86.54


In [19]:
trades_1.df.filter(pl.col("Gain/Loss%") < -70.54)

"364 days, 0:00:00_week_min_targets/exit_column",SMA100_slope_up,Symbol,Start,End,Entry_Price,Exit_Price,Low,High,Length,Gain/Loss,Gain/Loss%,Highest%,Lowest%
i32,bool,str,datetime[μs],datetime[μs],f64,f64,f64,f64,u32,f64,f64,f64,f64
40239,false,"""SPRO""",2022-04-26 00:00:00,2022-05-03 00:00:00,5.353,1.5,1.495,5.54,6,-3.853,-71.98,3.49,-72.07


In [16]:
trades_1.df.filter(pl.col("Highest%") == 557.16)

"364 days, 0:00:00_week_min_targets/exit_column",SMA100_slope_up,Symbol,Start,End,Entry_Price,Exit_Price,Low,High,Length,Gain/Loss,Gain/Loss%,Highest%,Lowest%
i32,bool,str,datetime[μs],datetime[μs],f64,f64,f64,f64,u32,f64,f64,f64,f64
30482,true,"""CARM""",2021-08-13 00:00:00,2021-08-13 00:00:00,18.382001,19.301101,14.0,120.800003,1,0.9191,5.0,557.16,-23.84


In [17]:
tf = trades_05.df.groupby(pi.indicators.SYMBOL_COLUMN).agg(pl.col(pi.indicators.SYMBOL_COLUMN).count().alias("trades")).sort("trades", descending=True)
tf

Symbol,trades
str,u32
"""ROOT""",65
"""AYTU""",65
"""BXRX""",60
"""PALI""",60
"""COMS""",60
"""WISA""",60
"""CZOO""",58
"""KALA""",57
"""NBY""",56


In [18]:
trades_05.df

"364 days, 0:00:00_week_min_targets/exit_column",SMA100_slope_up,Symbol,Start,End,Entry_Price,Exit_Price,Low,High,Length,Gain/Loss,Gain/Loss%,Highest%,Lowest%
i32,bool,str,datetime[μs],datetime[μs],f64,f64,f64,f64,u32,f64,f64,f64,f64
1,false,"""CEMI""",2021-05-25 00:00:00,2021-06-03 00:00:00,3.02505,3.17,2.98,3.3,7,0.14495,4.79,9.09,-1.49
2,false,"""BWXT""",2021-08-03 00:00:00,2021-08-03 00:00:00,53.2851,55.949355,52.470001,56.279999,1,2.664255,5.0,5.62,-1.53
3,false,"""BWXT""",2021-08-05 00:00:00,2021-08-16 00:00:00,52.732351,55.110001,52.310001,55.91,8,2.377649,4.51,6.03,-0.8
4,false,"""BWXT""",2021-11-02 00:00:00,2021-11-08 00:00:00,52.571551,54.0,50.82,55.529999,5,1.428449,2.72,5.63,-3.33
5,false,"""BWXT""",2021-11-18 00:00:00,2021-11-23 00:00:00,51.0741,51.200001,50.310001,52.18,4,0.125901,0.25,2.17,-1.5
6,false,"""BWXT""",2021-11-24 00:00:00,2021-11-26 00:00:00,50.561551,49.84,48.919998,51.509998,2,-0.721551,-1.43,1.88,-3.25
7,false,"""BWXT""",2021-11-29 00:00:00,2021-12-09 00:00:00,49.164598,47.889999,46.799999,49.75,9,-1.274599,-2.59,1.19,-4.81
8,false,"""BWXT""",2021-12-15 00:00:00,2021-12-20 00:00:00,47.033999,46.700001,45.5,48.73,4,-0.333998,-0.71,3.61,-3.26
9,false,"""BWXT""",2022-01-26 00:00:00,2022-01-26 00:00:00,45.7275,48.013875,45.23,48.299999,1,2.286375,5.0,5.63,-1.09


In [ ]:
index = "index"
t = df.with_row_count(index).filter(pl.col(index) < 10).select(pl.col(pi.indicators.DATE_COLUMN),pi.indicators.LOW_COLUMN)

In [33]:
t.with_columns(pl.col("Low").rolling_min(2).shift(3).alias("roll"))

Date,Low,roll
datetime[μs],f64,f64
2020-01-02 00:00:00,4.26,null
2020-01-03 00:00:00,4.78,null
2020-01-06 00:00:00,4.85,null
2020-01-07 00:00:00,4.82,null
2020-01-08 00:00:00,4.82,4.26
2020-01-09 00:00:00,4.85,4.78
2020-01-10 00:00:00,3.89,4.82
2020-01-13 00:00:00,4.21,4.82
2020-01-14 00:00:00,4.0,4.82


In [29]:
df.with_columns(pl.col(pi.indicators.LOW_COLUMN).arg_min().over(pi.indicators.SYMBOL_COLUMN).alias("tester"))

Date,Open,High,Low,Close,Adj Close,Volume,Symbol,SMA100,SMA100_slope_up,tester
datetime[μs],f64,f64,f64,f64,f64,i64,str,f64,bool,u32
2020-01-02 00:00:00,4.61,4.61,4.26,4.56,4.56,116200,"""CEMI""",null,null,753
2020-01-03 00:00:00,5.0,5.4,4.78,5.19,5.19,432600,"""CEMI""",null,null,753
2020-01-06 00:00:00,5.08,5.11,4.85,4.89,4.89,140400,"""CEMI""",null,null,753
2020-01-07 00:00:00,4.83,5.19,4.82,5.09,5.09,75200,"""CEMI""",null,null,753
2020-01-08 00:00:00,5.05,5.1,4.82,4.87,4.87,71300,"""CEMI""",null,null,753
2020-01-09 00:00:00,4.85,4.94,4.85,4.86,4.86,21500,"""CEMI""",null,null,753
2020-01-10 00:00:00,4.85,4.85,3.89,4.54,4.54,557100,"""CEMI""",null,null,753
2020-01-13 00:00:00,4.5,4.5,4.21,4.24,4.24,87500,"""CEMI""",null,null,753
2020-01-14 00:00:00,4.25,4.34,4.0,4.16,4.16,60800,"""CEMI""",null,null,753
